In [1]:
import boto3
import pandas as pd

s3 = boto3.client('s3')

s3 = boto3.resource(
    service_name='s3',
    region_name='us-east-1',
    aws_access_key_id='',
    aws_secret_access_key=''
)
# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


iibucketai
iibucketai-1


In [2]:
import os
os.environ["AWS_DEFAULT_REGION"] = ''
os.environ["AWS_ACCESS_KEY_ID"] = ''
os.environ["AWS_SECRET_ACCESS_KEY"] = ''

In [3]:
from io import BytesIO
import boto3
import numpy as np
train_x_obj = s3.Bucket('iibucketai').Object('train_x.npy').get()
train_y_obj = s3.Bucket('iibucketai').Object('train_y.npy').get()
train_x = np.load(BytesIO(train_x_obj['Body'].read()))
train_y = np.load(BytesIO(train_y_obj['Body'].read()))

test_x_obj = s3.Bucket('iibucketai').Object('test_x.npy').get()
test_y_obj = s3.Bucket('iibucketai').Object('test_y.npy').get()
test_x = np.load(BytesIO(test_x_obj['Body'].read()))
test_y = np.load(BytesIO(test_y_obj['Body'].read()))

val_x_obj = s3.Bucket('iibucketai').Object('val_x.npy').get()
val_y_obj = s3.Bucket('iibucketai').Object('val_y.npy').get()
val_x = np.load(BytesIO(val_x_obj['Body'].read()))
val_y = np.load(BytesIO(val_y_obj['Body'].read()))

In [4]:
print('Train : ',train_x.shape, train_y.shape)
print('Test : ',test_x.shape, test_y.shape)
print( 'Val : ',val_x.shape, val_y.shape)

Train :  (80000, 500) (80000, 500)
Test :  (80000, 500) (80000, 500)
Val :  (80000, 500) (80000, 500)


In [5]:
train_x = np.concatenate((train_x,val_x))

In [6]:
train_y = np.concatenate((train_y,val_y))

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
if len(train_x.shape)<3:
    train_X = train_x.reshape(train_x.shape[0],train_x.shape[1],1)
    train_y = train_y.reshape(train_y.shape[0],train_y.shape[1],1)
    val_X = val_x.reshape(val_x.shape[0],val_x.shape[1],1)
    test_X = test_x.reshape(test_x.shape[0],test_x.shape[1],1)
    #wy = y.reshape(y.shape[0],y.shape[1])

Input = keras.Input(shape=(500,1))
x = layers.Conv1D(8,3,activation='relu', padding = 'same')(Input)#(x)
x = layers.Conv1D(8,2,activation='relu', padding = 'same')(x)
x = layers.Conv1D(16,3,activation='relu', padding = 'same')(x)
x = layers.Conv1D(16,3,activation='relu', padding = 'same')(x)
x = layers.Conv1D(32,3,activation='relu', padding = 'same')(x)
x = layers.Conv1D(32,3,activation='relu', padding = 'same')(x)
x = layers.Conv1D(64,3,activation='relu', padding = 'same')(x)
x = layers.Conv1D(64,3,activation='relu', padding = 'same')(x)
x = layers.MaxPooling1D(2)(x)

x = layers.Flatten()(x)#(pool1)#(conv)

x = layers.Dense(500,activation='relu')(x)
x = layers.Dense(500,activation='relu')(x)
output = layers.Dense(500,activation='relu')(x)
model = keras.Model(inputs=Input, outputs=output, name = 'CNN')
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss=keras.losses.MSE,
                           metrics=['mae'])
model.summary()
history=model.fit(train_X,train_y,
                batch_size=128,epochs=100,
                #validation_data=(val_x,val_y),
                validation_split=0.3,
                shuffle=True)

Model: "CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 500, 1)]          0         
_________________________________________________________________
conv1d_28 (Conv1D)           (None, 500, 8)            32        
_________________________________________________________________
conv1d_29 (Conv1D)           (None, 500, 8)            136       
_________________________________________________________________
conv1d_30 (Conv1D)           (None, 500, 16)           400       
_________________________________________________________________
conv1d_31 (Conv1D)           (None, 500, 16)           784       
_________________________________________________________________
conv1d_32 (Conv1D)           (None, 500, 32)           1568      
_________________________________________________________________
conv1d_33 (Conv1D)           (None, 500, 32)           3104    